#### Imports





In [ ]:
%%capture
!pip install datasets
!pip install -U transformers
!apt install git-lfs

In [ ]:
import json
import torch
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, AutoTokenizer, BertTokenizerFast, BertForQuestionAnswering, AutoModelForQuestionAnswering
import transformers
from transformers import pipeline
from datasets import load_dataset, load_metric
from transformers import default_data_collator
from torch.utils.data import DataLoader
from transformers import get_scheduler
from transformers import AdamW
from tqdm.auto import tqdm
from transformers import AutoTokenizer


####  Data Loading.

In [ ]:
#loading cluster-wise csvs
%%capture
! gdown 1JaMYQrbtpREDw7E8VUqtNAPSod_eP9u2
! unzip datasets.zip

In [ ]:
def loaddataset(cluster = 0):
  base_url = 'datasets/'
  data_files = {"train":base_url + "{}_train.csv".format(cluster), "test": base_url + "{}_test.csv".format(cluster)}
  # data_files = {"train":base_url + "{}_train.csv".format(theme), "test": base_url + "{}_test.csv".format(theme)}
  dataset = load_dataset("csv", data_files=data_files)
  return dataset


#### Finetuning


##### Device Setup


In [ ]:
#setting up global variables
model_name = "PremalMatalia/electra-base-best-squad2"
batch_size = 8
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
data_collator = default_data_collator

In [ ]:
def setup_device(model_name):
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast), "Fast Tokenizer Needed"
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  pad_on_right = tokenizer.padding_side == "right"
  model.to(device)
  return model, tokenizer, pad_on_right



##### Preprocessing for FineTuning


In [ ]:
def preprocess_for_training(dataset, tokenizer, pad_on_right):
    dataset["Question"] = [q.lstrip() for q in dataset["Question"]]

    tokenized_examples = tokenizer(
        dataset["Question" if pad_on_right else "Paragraph"],
        dataset["Paragraph" if pad_on_right else "Qustion"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    # print(sample_mapping, offset_mapping)
    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        #! WORKS

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = {
            'text' : (dataset["Answer_text"][sample_index]),
            'answer_start' : [int(dataset["Answer_start"][sample_index][1:-1])] if dataset["Answer_start"][sample_index][1:-1] != '' else []
        }
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"]) - 4
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


##### Finetuning

In [ ]:
def fine_tune_using_api(model, tokenizer, pad_on_right, cluster = 0, learning_rate = 7e-7, epochs = 4, scheduler = 'linear'):
  dataset = loaddataset(cluster)
  #Training Args
  model_name_ = model_name.split("/")[-1]
  args = TrainingArguments(
      f"{model_name_}-finetuned-squad-{cluster}-trainedfor-{epochs}",
      evaluation_strategy = "epoch",
      learning_rate=learning_rate,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      lr_scheduler_type = "linear",
      optim = "adamw_hf",
      # warmup_proportion = 0.2,
      num_train_epochs=epochs,
      weight_decay=0.01,
      save_strategy = "no",
      push_to_hub=False, #because why not
  )
  tokenized_datasets = dataset.map(lambda x : preprocess_for_training(x, tokenizer, pad_on_right), batched=True, remove_columns=dataset["train"].column_names)
  trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
  )
  trainer.train()
  # login_please()
  trainer.save_model(f"{model_name_}-finetuned-squad-{cluster}")
  return model #interim, do not want to refactor all my code lmaooo